In [4]:
pip install mwparserfromhell

     |████████████████████████████████| 135 kB 824 kB/s 
  Created wheel for mwparserfromhell: filename=mwparserfromhell-0.5.4-cp38-cp38-macosx_10_9_x86_64.whl size=94282 sha256=00c8f0427e1102240998bf83c6773d8e1df19ca4ef2b789a2ee1c0858f32175c
  Stored in directory: /Users/ian/Library/Caches/pip/wheels/f9/c0/76/e38181ee594907b9a99ca0d30652ca8baae1f308052f810f79
Successfully built mwparserfromhell
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import time, os
from fake_useragent import UserAgent
import datetime
from tqdm import tqdm
import pickle
import xml.sax
import subprocess
import mwparserfromhell

In [2]:
from keras.utils import get_file

In [3]:
dump_url = base_url + '20201101/'
dump_html = requests.get(dump_url).text
soup_dump = BeautifulSoup(dump_html, 'html.parser')
soup_dump.find_all('li', {'class': 'file'})[:3]

[<li class="file"><a href="/enwiki/20201101/enwiki-20201101-pages-articles-multistream.xml.bz2">enwiki-20201101-pages-articles-multistream.xml.bz2</a> 17.6 GB</li>,
 <li class="file"><a href="/enwiki/20201101/enwiki-20201101-pages-articles-multistream-index.txt.bz2">enwiki-20201101-pages-articles-multistream-index.txt.bz2</a> 216.6 MB</li>,
 <li class="file"><a href="/enwiki/20201101/enwiki-20201101-pages-articles-multistream1.xml-p1p41242.bz2">enwiki-20201101-pages-articles-multistream1.xml-p1p41242.bz2</a> 232.4 MB</li>]

In [4]:
%%timeit
for wiki_file in tqdm(files):
    get_file(wiki_file, f'https://dumps.wikimedia.org/enwiki/20201101/{wiki_file}')

.wikimedia.org/enwiki/20201101/enwiki-20201101-pages-articles-multistream-index15.txt-p17324603p17460152.bz2
100%|██████████| 118/118 [00:00<00:00, 8468.41it/s]15.2 ms ± 1.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)



In [17]:
def process_article(title, text, template='Infobox settlement'):
    """Process a Wikipedia article looking for template"""
    
    # Create a parsing object
    wikicode = mwparserfromhell.parse(text)
    
    # Search through templates for the template
    matches = wikicode.filter_templates(matches=template)
    
    if len(matches) >= 1:
        # Extract information from infobox
        properties = {param.name.strip_code().strip(): param.value.strip_code().strip() 
                      for param in matches[0].params
                      if param.value.strip_code().strip()}
        
        # Extract internal wikilinks
        wikilinks = [x.title.strip_code().strip() for x in wikicode.filter_wikilinks()]
        # Extract external links
        exlinks = [x.url.strip_code().strip() for x in wikicode.filter_external_links()]
        return (title, properties, wikilinks, exlinks)

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []
        self._article_count = 0
        self._places = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)
        if name == 'page':
            self._article_count += 1
            # Send the page to the process_article function
            place = process_article(**self._values, 
                                    template='Infobox settlement')
            # If article is a place append to the list of places
            if place:
                self._places.append(place)

In [7]:
ls -t

enwiki-20201101-pages-articles-multistream.xml.bz2
xowa_app_macosx_64_v4.6.12.2009/
xowa_app_macosx_64_v4.6.12.2009.zip
Number_of_restaurants_per_100,000_population_23062020.csv
archive (5)/
archive (5).zip
archive (4) 2/
archive (4).zip
Definitions of 5-Scale Student Academic Satisfactory Progress-1.pdf
Alaska shape (1).png
Alaska shape.png
Alaska.geojson
mygeodata.zip
List_of_national_and_international_statistical_services.html
Turkeys.png
bbc.zip
Image from iOS.png
Clusta_Rhymes.pdf
TableauAutoUpdate/
motivational-eventwav_Gy-BJISO.mp3
ships (1).dta
ships.dta
Table2.xls
World_Travel_Advisories.csv
Court 1 smaller.png
Court 2.png
Court 1.webp
NBA project test (1)/
NBA project test (1).zip
NBA project test.zip
Court.png
Giannis yellow.png
Giannis vector (1).png
Giannis vector.png
Basketball.png
IMG_4097.jpg
IMG_4094.HEIC
IMG_4093.jpg
IMG_4092.jpg
IMG_4091.jpg
IMG_4090.jpg
FileZilla_3.50.0_macosx-x86.app.tar.bz2
abalone.names
abalone.data
download.html
archive (3)/
archive (3).zip
arch

In [18]:
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)
data_path = 'enwiki-20201101-pages-articles-multistream.xml.bz2'

for line in tqdm(subprocess.Popen(['bzcat'], stdin=open(data_path), stdout=subprocess.PIPE).stdout):
    try:
        parser.feed(line)
    except StopIteration:
        break

1104629346it [29:40:25, 10340.48it/s]


KeyboardInterrupt: 

In [ ]:
handler

In [ ]:
import mwparserfromhell

# Create the wiki article
wiki = mwparserfromhell.parse(handler._pages[6][1])

# Find the wikilinks
wikilinks = [x.title for x in wiki.filter_wikilinks()]
wikilinks[:5]

In [9]:
bz2_files = ["enwiki-20201101-pages-articles-multistream1.xml-p1p41242.bz2", 
"enwiki-20201101-pages-articles-multistream-index1.txt-p1p41242.bz2", 
"enwiki-20201101-pages-articles-multistream2.xml-p41243p151573.bz2", 
"enwiki-20201101-pages-articles-multistream-index2.txt-p41243p151573.bz2", 
"enwiki-20201101-pages-articles-multistream3.xml-p151574p311329.bz2", 
"enwiki-20201101-pages-articles-multistream-index3.txt-p151574p311329.bz2", 
"enwiki-20201101-pages-articles-multistream4.xml-p311330p558391.bz2", 
"enwiki-20201101-pages-articles-multistream-index4.txt-p311330p558391.bz2", 
"enwiki-20201101-pages-articles-multistream5.xml-p558392p958045.bz2", 
"enwiki-20201101-pages-articles-multistream-index5.txt-p558392p958045.bz2", 
"enwiki-20201101-pages-articles-multistream6.xml-p958046p1483661.bz2", 
"enwiki-20201101-pages-articles-multistream-index6.txt-p958046p1483661.bz2", 
"enwiki-20201101-pages-articles-multistream7.xml-p1483662p2134111.bz2", 
"enwiki-20201101-pages-articles-multistream-index7.txt-p1483662p2134111.bz2", 
"enwiki-20201101-pages-articles-multistream8.xml-p2134112p2936260.bz2", 
"enwiki-20201101-pages-articles-multistream-index8.txt-p2134112p2936260.bz2", 
"enwiki-20201101-pages-articles-multistream9.xml-p2936261p4045402.bz2", 
"enwiki-20201101-pages-articles-multistream-index9.txt-p2936261p4045402.bz2", 
"enwiki-20201101-pages-articles-multistream10.xml-p4045403p5399366.bz2", 
"enwiki-20201101-pages-articles-multistream-index10.txt-p4045403p5399366.bz2", 
"enwiki-20201101-pages-articles-multistream11.xml-p5399367p6899366.bz2", 
"enwiki-20201101-pages-articles-multistream-index11.txt-p5399367p6899366.bz2", 
"enwiki-20201101-pages-articles-multistream11.xml-p6899367p7054859.bz2", 
"enwiki-20201101-pages-articles-multistream-index11.txt-p6899367p7054859.bz2", 
"enwiki-20201101-pages-articles-multistream12.xml-p7054860p8554859.bz2", 
"enwiki-20201101-pages-articles-multistream-index12.txt-p7054860p8554859.bz2", 
"enwiki-20201101-pages-articles-multistream12.xml-p8554860p9172788.bz2", 
"enwiki-20201101-pages-articles-multistream-index12.txt-p8554860p9172788.bz2", 
"enwiki-20201101-pages-articles-multistream13.xml-p9172789p10672788.bz2", 
"enwiki-20201101-pages-articles-multistream-index13.txt-p9172789p10672788.bz2", 
"enwiki-20201101-pages-articles-multistream13.xml-p10672789p11659682.bz2", 
"enwiki-20201101-pages-articles-multistream-index13.txt-p10672789p11659682.bz2", 
"enwiki-20201101-pages-articles-multistream14.xml-p11659683p13159682.bz2", 
"enwiki-20201101-pages-articles-multistream-index14.txt-p11659683p13159682.bz2", 
"enwiki-20201101-pages-articles-multistream14.xml-p13159683p14324602.bz2", 
"enwiki-20201101-pages-articles-multistream-index14.txt-p13159683p14324602.bz2", 
"enwiki-20201101-pages-articles-multistream15.xml-p14324603p15824602.bz2", 
"enwiki-20201101-pages-articles-multistream-index15.txt-p14324603p15824602.bz2", 
"enwiki-20201101-pages-articles-multistream15.xml-p15824603p17324602.bz2", 
"enwiki-20201101-pages-articles-multistream-index15.txt-p15824603p17324602.bz2", 
"enwiki-20201101-pages-articles-multistream15.xml-p17324603p17460152.bz2", 
"enwiki-20201101-pages-articles-multistream-index15.txt-p17324603p17460152.bz2", 
"enwiki-20201101-pages-articles-multistream16.xml-p17460153p18960152.bz2", 
"enwiki-20201101-pages-articles-multistream-index16.txt-p17460153p18960152.bz2", 
"enwiki-20201101-pages-articles-multistream16.xml-p18960153p20460152.bz2", 
"enwiki-20201101-pages-articles-multistream-index16.txt-p18960153p20460152.bz2", 
"enwiki-20201101-pages-articles-multistream16.xml-p20460153p20570392.bz2", 
"enwiki-20201101-pages-articles-multistream-index16.txt-p20460153p20570392.bz2", 
"enwiki-20201101-pages-articles-multistream17.xml-p20570393p22070392.bz2", 
"enwiki-20201101-pages-articles-multistream-index17.txt-p20570393p22070392.bz2", 
"enwiki-20201101-pages-articles-multistream17.xml-p22070393p23570392.bz2", 
"enwiki-20201101-pages-articles-multistream-index17.txt-p22070393p23570392.bz2", 
"enwiki-20201101-pages-articles-multistream17.xml-p23570393p23716197.bz2", 
"enwiki-20201101-pages-articles-multistream-index17.txt-p23570393p23716197.bz2", 
"enwiki-20201101-pages-articles-multistream18.xml-p23716198p25216197.bz2", 
"enwiki-20201101-pages-articles-multistream-index18.txt-p23716198p25216197.bz2", 
"enwiki-20201101-pages-articles-multistream18.xml-p25216198p26716197.bz2", 
"enwiki-20201101-pages-articles-multistream-index18.txt-p25216198p26716197.bz2", 
"enwiki-20201101-pages-articles-multistream18.xml-p26716198p27121850.bz2", 
"enwiki-20201101-pages-articles-multistream-index18.txt-p26716198p27121850.bz2", 
"enwiki-20201101-pages-articles-multistream19.xml-p27121851p28621850.bz2", 
"enwiki-20201101-pages-articles-multistream-index19.txt-p27121851p28621850.bz2", 
"enwiki-20201101-pages-articles-multistream19.xml-p28621851p30121850.bz2", 
"enwiki-20201101-pages-articles-multistream-index19.txt-p28621851p30121850.bz2", 
"enwiki-20201101-pages-articles-multistream19.xml-p30121851p31308442.bz2", 
"enwiki-20201101-pages-articles-multistream-index19.txt-p30121851p31308442.bz2", 
"enwiki-20201101-pages-articles-multistream20.xml-p31308443p32808442.bz2", 
"enwiki-20201101-pages-articles-multistream-index20.txt-p31308443p32808442.bz2", 
"enwiki-20201101-pages-articles-multistream20.xml-p32808443p34308442.bz2", 
"enwiki-20201101-pages-articles-multistream-index20.txt-p32808443p34308442.bz2", 
"enwiki-20201101-pages-articles-multistream20.xml-p34308443p35522432.bz2", 
"enwiki-20201101-pages-articles-multistream-index20.txt-p34308443p35522432.bz2", 
"enwiki-20201101-pages-articles-multistream21.xml-p35522433p37022432.bz2", 
"enwiki-20201101-pages-articles-multistream-index21.txt-p35522433p37022432.bz2", 
"enwiki-20201101-pages-articles-multistream21.xml-p37022433p38522432.bz2", 
"enwiki-20201101-pages-articles-multistream-index21.txt-p37022433p38522432.bz2", 
"enwiki-20201101-pages-articles-multistream21.xml-p38522433p39996245.bz2", 
"enwiki-20201101-pages-articles-multistream-index21.txt-p38522433p39996245.bz2", 
"enwiki-20201101-pages-articles-multistream22.xml-p39996246p41496245.bz2", 
"enwiki-20201101-pages-articles-multistream-index22.txt-p39996246p41496245.bz2", 
"enwiki-20201101-pages-articles-multistream22.xml-p41496246p42996245.bz2", 
"enwiki-20201101-pages-articles-multistream-index22.txt-p41496246p42996245.bz2", 
"enwiki-20201101-pages-articles-multistream22.xml-p42996246p44496245.bz2", 
"enwiki-20201101-pages-articles-multistream-index22.txt-p42996246p44496245.bz2", 
"enwiki-20201101-pages-articles-multistream22.xml-p44496246p44788941.bz2", 
"enwiki-20201101-pages-articles-multistream-index22.txt-p44496246p44788941.bz2", 
"enwiki-20201101-pages-articles-multistream23.xml-p44788942p46288941.bz2", 
"enwiki-20201101-pages-articles-multistream-index23.txt-p44788942p46288941.bz2", 
"enwiki-20201101-pages-articles-multistream23.xml-p46288942p47788941.bz2", 
"enwiki-20201101-pages-articles-multistream-index23.txt-p46288942p47788941.bz2", 
"enwiki-20201101-pages-articles-multistream23.xml-p47788942p49288941.bz2", 
"enwiki-20201101-pages-articles-multistream-index23.txt-p47788942p49288941.bz2", 
"enwiki-20201101-pages-articles-multistream23.xml-p49288942p50564553.bz2", 
"enwiki-20201101-pages-articles-multistream-index23.txt-p49288942p50564553.bz2", 
"enwiki-20201101-pages-articles-multistream24.xml-p50564554p52064553.bz2", 
"enwiki-20201101-pages-articles-multistream-index24.txt-p50564554p52064553.bz2", 
"enwiki-20201101-pages-articles-multistream24.xml-p52064554p53564553.bz2", 
"enwiki-20201101-pages-articles-multistream-index24.txt-p52064554p53564553.bz2", 
"enwiki-20201101-pages-articles-multistream24.xml-p53564554p55064553.bz2", 
"enwiki-20201101-pages-articles-multistream-index24.txt-p53564554p55064553.bz2", 
"enwiki-20201101-pages-articles-multistream24.xml-p55064554p56564553.bz2", 
"enwiki-20201101-pages-articles-multistream-index24.txt-p55064554p56564553.bz2", 
"enwiki-20201101-pages-articles-multistream24.xml-p56564554p57025655.bz2", 
"enwiki-20201101-pages-articles-multistream-index24.txt-p56564554p57025655.bz2", 
"enwiki-20201101-pages-articles-multistream25.xml-p57025656p58525655.bz2", 
"enwiki-20201101-pages-articles-multistream-index25.txt-p57025656p58525655.bz2", 
"enwiki-20201101-pages-articles-multistream25.xml-p58525656p60025655.bz2", 
"enwiki-20201101-pages-articles-multistream-index25.txt-p58525656p60025655.bz2", 
"enwiki-20201101-pages-articles-multistream25.xml-p60025656p61525655.bz2", 
"enwiki-20201101-pages-articles-multistream-index25.txt-p60025656p61525655.bz2", 
"enwiki-20201101-pages-articles-multistream25.xml-p61525656p62585850.bz2", 
"enwiki-20201101-pages-articles-multistream-index25.txt-p61525656p62585850.bz2", 
"enwiki-20201101-pages-articles-multistream26.xml-p62585851p63975909.bz2", 
"enwiki-20201101-pages-articles-multistream-index26.txt-p62585851p63975909.bz2", 
"enwiki-20201101-pages-articles-multistream27.xml-p63975910p65475909.bz2", 
"enwiki-20201101-pages-articles-multistream-index27.txt-p63975910p65475909.bz2", 
"enwiki-20201101-pages-articles-multistream27.xml-p65475910p65740670.bz2", 
"enwiki-20201101-pages-articles-multistream-index27.txt-p65475910p65740670.bz2"]

In [3]:
files = ["enwiki-20201101-pages-articles-multistream1.xml-p1p41242.bz2", 
"enwiki-20201101-pages-articles-multistream-index1.txt-p1p41242.bz2", 
"enwiki-20201101-pages-articles-multistream2.xml-p41243p151573.bz2", 
"enwiki-20201101-pages-articles-multistream-index2.txt-p41243p151573.bz2", 
"enwiki-20201101-pages-articles-multistream3.xml-p151574p311329.bz2", 
"enwiki-20201101-pages-articles-multistream-index3.txt-p151574p311329.bz2", 
"enwiki-20201101-pages-articles-multistream4.xml-p311330p558391.bz2", 
"enwiki-20201101-pages-articles-multistream-index4.txt-p311330p558391.bz2", 
"enwiki-20201101-pages-articles-multistream5.xml-p558392p958045.bz2", 
"enwiki-20201101-pages-articles-multistream-index5.txt-p558392p958045.bz2", 
"enwiki-20201101-pages-articles-multistream6.xml-p958046p1483661.bz2", 
"enwiki-20201101-pages-articles-multistream-index6.txt-p958046p1483661.bz2", 
"enwiki-20201101-pages-articles-multistream7.xml-p1483662p2134111.bz2", 
"enwiki-20201101-pages-articles-multistream-index7.txt-p1483662p2134111.bz2", 
"enwiki-20201101-pages-articles-multistream8.xml-p2134112p2936260.bz2", 
"enwiki-20201101-pages-articles-multistream-index8.txt-p2134112p2936260.bz2", 
"enwiki-20201101-pages-articles-multistream9.xml-p2936261p4045402.bz2", 
"enwiki-20201101-pages-articles-multistream-index9.txt-p2936261p4045402.bz2", 
"enwiki-20201101-pages-articles-multistream10.xml-p4045403p5399366.bz2", "enwiki-20201101-pages-articles-multistream-index10.txt-p4045403p5399366.bz2", "enwiki-20201101-pages-articles-multistream11.xml-p5399367p6899366.bz2", "enwiki-20201101-pages-articles-multistream-index11.txt-p5399367p6899366.bz2", "enwiki-20201101-pages-articles-multistream11.xml-p6899367p7054859.bz2", "enwiki-20201101-pages-articles-multistream-index11.txt-p6899367p7054859.bz2", "enwiki-20201101-pages-articles-multistream12.xml-p7054860p8554859.bz2", "enwiki-20201101-pages-articles-multistream-index12.txt-p7054860p8554859.bz2", "enwiki-20201101-pages-articles-multistream12.xml-p8554860p9172788.bz2", "enwiki-20201101-pages-articles-multistream-index12.txt-p8554860p9172788.bz2", "enwiki-20201101-pages-articles-multistream13.xml-p9172789p10672788.bz2", "enwiki-20201101-pages-articles-multistream-index13.txt-p9172789p10672788.bz2", "enwiki-20201101-pages-articles-multistream13.xml-p10672789p11659682.bz2", "enwiki-20201101-pages-articles-multistream-index13.txt-p10672789p11659682.bz2", "enwiki-20201101-pages-articles-multistream14.xml-p11659683p13159682.bz2", "enwiki-20201101-pages-articles-multistream-index14.txt-p11659683p13159682.bz2", "enwiki-20201101-pages-articles-multistream14.xml-p13159683p14324602.bz2", "enwiki-20201101-pages-articles-multistream-index14.txt-p13159683p14324602.bz2", "enwiki-20201101-pages-articles-multistream15.xml-p14324603p15824602.bz2", "enwiki-20201101-pages-articles-multistream-index15.txt-p14324603p15824602.bz2", "enwiki-20201101-pages-articles-multistream15.xml-p15824603p17324602.bz2", "enwiki-20201101-pages-articles-multistream-index15.txt-p15824603p17324602.bz2", "enwiki-20201101-pages-articles-multistream15.xml-p17324603p17460152.bz2", "enwiki-20201101-pages-articles-multistream-index15.txt-p17324603p17460152.bz2", "enwiki-20201101-pages-articles-multistream16.xml-p17460153p18960152.bz2", "enwiki-20201101-pages-articles-multistream-index16.txt-p17460153p18960152.bz2", "enwiki-20201101-pages-articles-multistream16.xml-p18960153p20460152.bz2", "enwiki-20201101-pages-articles-multistream-index16.txt-p18960153p20460152.bz2", "enwiki-20201101-pages-articles-multistream16.xml-p20460153p20570392.bz2", "enwiki-20201101-pages-articles-multistream-index16.txt-p20460153p20570392.bz2", "enwiki-20201101-pages-articles-multistream17.xml-p20570393p22070392.bz2", "enwiki-20201101-pages-articles-multistream-index17.txt-p20570393p22070392.bz2", "enwiki-20201101-pages-articles-multistream17.xml-p22070393p23570392.bz2", "enwiki-20201101-pages-articles-multistream-index17.txt-p22070393p23570392.bz2", "enwiki-20201101-pages-articles-multistream17.xml-p23570393p23716197.bz2", "enwiki-20201101-pages-articles-multistream-index17.txt-p23570393p23716197.bz2", "enwiki-20201101-pages-articles-multistream18.xml-p23716198p25216197.bz2", "enwiki-20201101-pages-articles-multistream-index18.txt-p23716198p25216197.bz2", "enwiki-20201101-pages-articles-multistream18.xml-p25216198p26716197.bz2", "enwiki-20201101-pages-articles-multistream-index18.txt-p25216198p26716197.bz2", "enwiki-20201101-pages-articles-multistream18.xml-p26716198p27121850.bz2", "enwiki-20201101-pages-articles-multistream-index18.txt-p26716198p27121850.bz2", "enwiki-20201101-pages-articles-multistream19.xml-p27121851p28621850.bz2", "enwiki-20201101-pages-articles-multistream-index19.txt-p27121851p28621850.bz2", "enwiki-20201101-pages-articles-multistream19.xml-p28621851p30121850.bz2", "enwiki-20201101-pages-articles-multistream-index19.txt-p28621851p30121850.bz2", "enwiki-20201101-pages-articles-multistream19.xml-p30121851p31308442.bz2", "enwiki-20201101-pages-articles-multistream-index19.txt-p30121851p31308442.bz2", "enwiki-20201101-pages-articles-multistream20.xml-p31308443p32808442.bz2", "enwiki-20201101-pages-articles-multistream-index20.txt-p31308443p32808442.bz2", "enwiki-20201101-pages-articles-multistream20.xml-p32808443p34308442.bz2", "enwiki-20201101-pages-articles-multistream-index20.txt-p32808443p34308442.bz2", "enwiki-20201101-pages-articles-multistream20.xml-p34308443p35522432.bz2", "enwiki-20201101-pages-articles-multistream-index20.txt-p34308443p35522432.bz2", "enwiki-20201101-pages-articles-multistream21.xml-p35522433p37022432.bz2", "enwiki-20201101-pages-articles-multistream-index21.txt-p35522433p37022432.bz2", "enwiki-20201101-pages-articles-multistream21.xml-p37022433p38522432.bz2", "enwiki-20201101-pages-articles-multistream-index21.txt-p37022433p38522432.bz2", "enwiki-20201101-pages-articles-multistream21.xml-p38522433p39996245.bz2", "enwiki-20201101-pages-articles-multistream-index21.txt-p38522433p39996245.bz2", "enwiki-20201101-pages-articles-multistream22.xml-p39996246p41496245.bz2", "enwiki-20201101-pages-articles-multistream-index22.txt-p39996246p41496245.bz2", "enwiki-20201101-pages-articles-multistream22.xml-p41496246p42996245.bz2", "enwiki-20201101-pages-articles-multistream-index22.txt-p41496246p42996245.bz2", "enwiki-20201101-pages-articles-multistream22.xml-p42996246p44496245.bz2", "enwiki-20201101-pages-articles-multistream-index22.txt-p42996246p44496245.bz2", "enwiki-20201101-pages-articles-multistream22.xml-p44496246p44788941.bz2", "enwiki-20201101-pages-articles-multistream-index22.txt-p44496246p44788941.bz2", "enwiki-20201101-pages-articles-multistream23.xml-p44788942p46288941.bz2", "enwiki-20201101-pages-articles-multistream-index23.txt-p44788942p46288941.bz2", "enwiki-20201101-pages-articles-multistream23.xml-p46288942p47788941.bz2", "enwiki-20201101-pages-articles-multistream-index23.txt-p46288942p47788941.bz2", "enwiki-20201101-pages-articles-multistream23.xml-p47788942p49288941.bz2", "enwiki-20201101-pages-articles-multistream-index23.txt-p47788942p49288941.bz2", "enwiki-20201101-pages-articles-multistream23.xml-p49288942p50564553.bz2", "enwiki-20201101-pages-articles-multistream-index23.txt-p49288942p50564553.bz2", "enwiki-20201101-pages-articles-multistream24.xml-p50564554p52064553.bz2", "enwiki-20201101-pages-articles-multistream-index24.txt-p50564554p52064553.bz2", "enwiki-20201101-pages-articles-multistream24.xml-p52064554p53564553.bz2", "enwiki-20201101-pages-articles-multistream-index24.txt-p52064554p53564553.bz2", "enwiki-20201101-pages-articles-multistream24.xml-p53564554p55064553.bz2", "enwiki-20201101-pages-articles-multistream-index24.txt-p53564554p55064553.bz2", "enwiki-20201101-pages-articles-multistream24.xml-p55064554p56564553.bz2", "enwiki-20201101-pages-articles-multistream-index24.txt-p55064554p56564553.bz2", "enwiki-20201101-pages-articles-multistream24.xml-p56564554p57025655.bz2", "enwiki-20201101-pages-articles-multistream-index24.txt-p56564554p57025655.bz2", "enwiki-20201101-pages-articles-multistream25.xml-p57025656p58525655.bz2", "enwiki-20201101-pages-articles-multistream-index25.txt-p57025656p58525655.bz2", "enwiki-20201101-pages-articles-multistream25.xml-p58525656p60025655.bz2", "enwiki-20201101-pages-articles-multistream-index25.txt-p58525656p60025655.bz2", "enwiki-20201101-pages-articles-multistream25.xml-p60025656p61525655.bz2", "enwiki-20201101-pages-articles-multistream-index25.txt-p60025656p61525655.bz2", "enwiki-20201101-pages-articles-multistream25.xml-p61525656p62585850.bz2", "enwiki-20201101-pages-articles-multistream-index25.txt-p61525656p62585850.bz2", "enwiki-20201101-pages-articles-multistream26.xml-p62585851p63975909.bz2", "enwiki-20201101-pages-articles-multistream-index26.txt-p62585851p63975909.bz2", "enwiki-20201101-pages-articles-multistream27.xml-p63975910p65475909.bz2", "enwiki-20201101-pages-articles-multistream-index27.txt-p63975910p65475909.bz2", "enwiki-20201101-pages-articles-multistream27.xml-p65475910p65740670.bz2", "enwiki-20201101-pages-articles-multistream-index27.txt-p65475910p65740670.bz2"]